# 📊 Notebook 05 - Reporte Final

## Portfolio Construction via Clustering - Consolidación de Resultados

**Autor:** Juan Carlos Ruiz Arteaga  
**Fecha:** Enero 2026  
**Versión:** 1.0

---

### Contenido:
1. Resumen Ejecutivo
2. Datos Utilizados
3. Resultados del Clustering
4. Perfiles de Inversión y Composición de Carteras
5. Resultados del Backtesting
6. Comparación de Perfiles
7. Conclusiones y Recomendaciones
8. Disclaimer Legal

## 1. Configuración e Importación de Librerías

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from IPython.display import display, HTML, Markdown
import warnings
warnings.filterwarnings('ignore')

# Configuración de rutas
PATH_SEGMENTACION = r'C:\Users\carlo\Documents\4.DS\riskmanagement2025\data\segmentacion_final'
PATH_REPORTS = r'C:\Users\carlo\Documents\4.DS\riskmanagement2025\reports'
PATH_DATA = r'C:\Users\carlo\Documents\4.DS\riskmanagement2025\data'

print("✅ Librerías cargadas correctamente")
print(f"📁 Directorio de datos: {PATH_DATA}")
print(f"📁 Directorio de reportes: {PATH_REPORTS}")

✅ Librerías cargadas correctamente
📁 Directorio de datos: C:\Users\carlo\Documents\4.DS\riskmanagement2025\data
📁 Directorio de reportes: C:\Users\carlo\Documents\4.DS\riskmanagement2025\reports


## 2. Resumen Ejecutivo

In [17]:
resumen_html = """
<div style="background: linear-gradient(135deg, #1a1a2e 0%, #16213e 100%); padding: 30px; border-radius: 15px; color: white; font-family: Arial;">
    <h2 style="text-align: center; color: #00d4ff; margin-bottom: 25px;">📊 RESUMEN EJECUTIVO</h2>
    
    <div style="display: flex; justify-content: space-around; flex-wrap: wrap;">
        <div style="background: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px; margin: 10px; min-width: 200px; text-align: center;">
            <h3 style="color: #00ff88; margin: 0;">468</h3>
            <p style="margin: 5px 0 0 0;">Activos Analizados</p>
        </div>
        <div style="background: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px; margin: 10px; min-width: 200px; text-align: center;">
            <h3 style="color: #00ff88; margin: 0;">4 Años</h3>
            <p style="margin: 5px 0 0 0;">Datos Históricos (2019-2023)</p>
        </div>
        <div style="background: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px; margin: 10px; min-width: 200px; text-align: center;">
            <h3 style="color: #00ff88; margin: 0;">5</h3>
            <p style="margin: 5px 0 0 0;">Segmentos Identificados</p>
        </div>
        <div style="background: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px; margin: 10px; min-width: 200px; text-align: center;">
            <h3 style="color: #00ff88; margin: 0;">5</h3>
            <p style="margin: 5px 0 0 0;">Perfiles de Inversión</p>
        </div>
    </div>
    
    <div style="margin-top: 25px; padding: 20px; background: rgba(0,212,255,0.1); border-radius: 10px; border-left: 4px solid #00d4ff;">
        <h4 style="color: #00d4ff; margin-top: 0;">🎯 Objetivo del Proyecto</h4>
        <p>Construcción de carteras de inversión diversificadas utilizando técnicas de <strong>clustering no supervisado</strong> 
        sobre activos financieros (acciones y ETFs), con validación mediante <strong>backtesting</strong> 
        contra el benchmark <strong>SPY (S&P 500)</strong>.</p>
    </div>
    
    <div style="margin-top: 20px; padding: 20px; background: rgba(0,255,136,0.1); border-radius: 10px; border-left: 4px solid #00ff88;">
        <h4 style="color: #00ff88; margin-top: 0;">✅ Resultado Principal</h4>
        <p>Las carteras construidas mediante el algoritmo de selección por <strong>Score de Momentum</strong> 
        lograron superar al benchmark SPY en el período de prueba (2024-2025), generando <strong>alpha positivo</strong> 
        en múltiples perfiles de inversión.</p>
    </div>
</div>
"""

display(HTML(resumen_html))

## 3. Carga de Datos Consolidados

In [3]:
# Cargar datos de segmentación
df_segmentacion = pd.read_csv(f'{PATH_SEGMENTACION}/activos_segmentados_kmeans.csv')
df_resumen_segmentos = pd.read_csv(f'{PATH_SEGMENTACION}/resumen_segmentos.csv')

# Cargar datos de precios
df_precios_train = pd.read_csv(f'{PATH_DATA}/prices_train.csv', parse_dates=['date'], index_col='date')
df_precios_test = pd.read_csv(f'{PATH_DATA}/prices_test.csv', parse_dates=['date'], index_col='date')

# Cargar portafolios de cada perfil
perfiles = ['conservador', 'moderado', 'agresivo', 'especulativo', 'normal']
portafolios = {}
metricas_backtest = {}
equity_curves = {}

for perfil in perfiles:
    try:
        portafolios[perfil] = pd.read_csv(f'{PATH_REPORTS}/portafolio_{perfil}.csv')
        metricas_backtest[perfil] = pd.read_csv(f'{PATH_REPORTS}/backtest_metricas_{perfil}.csv')
        # Cargar equity curves (columna 'Fecha' en lugar de 'date')
        df_equity = pd.read_csv(f'{PATH_REPORTS}/backtest_equity_curves_{perfil}.csv', parse_dates=['Fecha'], index_col='Fecha')
        # Renombrar columnas para consistencia
        df_equity.columns = ['equity_portafolio', 'equity_benchmark']
        equity_curves[perfil] = df_equity
    except FileNotFoundError:
        print(f"⚠️ No se encontraron datos para el perfil: {perfil}")
    except Exception as e:
        print(f"⚠️ Error cargando {perfil}: {e}")

print(f"\n✅ Datos cargados exitosamente")
print(f"📊 Activos en segmentación: {len(df_segmentacion)}")
print(f"📈 Período de entrenamiento: {df_precios_train.index.min().strftime('%Y-%m-%d')} a {df_precios_train.index.max().strftime('%Y-%m-%d')}")
print(f"📈 Período de prueba: {df_precios_test.index.min().strftime('%Y-%m-%d')} a {df_precios_test.index.max().strftime('%Y-%m-%d')}")
print(f"📋 Perfiles cargados: {list(portafolios.keys())}")


✅ Datos cargados exitosamente
📊 Activos en segmentación: 468
📈 Período de entrenamiento: 2021-01-04 a 2023-12-29
📈 Período de prueba: 2024-01-02 a 2025-12-19
📋 Perfiles cargados: ['conservador', 'moderado', 'agresivo', 'especulativo', 'normal']


## 4. Resultados del Clustering

In [4]:
# Tabla de distribución de segmentos
print("="*80)
print("📊 DISTRIBUCIÓN DE ACTIVOS POR SEGMENTO")
print("="*80)

# Crear resumen de segmentos con estadísticas clave
segmentos_resumen = df_segmentacion.groupby(['segmento', 'segmento_nombre']).agg({
    'ticker': 'count',
    'return_annualized': ['mean', 'std'],
    'volatility_annual': 'mean',
    'sharpe_ratio': 'mean',
    'beta': 'mean',
    'max_drawdown': 'mean'
}).round(4)

segmentos_resumen.columns = ['Activos', 'Ret. Medio', 'Ret. Std', 'Volatilidad', 'Sharpe', 'Beta', 'Max DD']
segmentos_resumen = segmentos_resumen.reset_index()

# Formatear para display
display(segmentos_resumen.style.format({
    'Ret. Medio': '{:.2%}',
    'Ret. Std': '{:.2%}',
    'Volatilidad': '{:.2%}',
    'Sharpe': '{:.2f}',
    'Beta': '{:.2f}',
    'Max DD': '{:.2%}'
}).background_gradient(subset=['Sharpe'], cmap='RdYlGn'))

📊 DISTRIBUCIÓN DE ACTIVOS POR SEGMENTO


,segmento,segmento_nombre,Activos,Ret. Medio,Ret. Std,Volatilidad,Sharpe,Beta,Max DD
0,-1,Outliers_Riesgo_Extremo,29,-8.84%,35.78%,57.79%,-0.07,0.92,-62.68%
1,0,Conservador,50,-8.24%,4.96%,22.80%,-0.55,0.56,-43.19%
2,1,Alto_Rendimiento,96,23.11%,8.64%,30.71%,0.66,0.98,-33.33%
3,2,Moderado,94,0.43%,8.87%,38.84%,0.06,1.30,-56.82%
4,3,Estable,199,6.12%,4.42%,23.66%,0.15,0.76,-32.64%


In [5]:
# Gráfico de distribución de segmentos (Pie Chart)
conteo_segmentos = df_segmentacion.groupby('segmento_nombre').size().reset_index(name='count')

colores_segmentos = {
    'Outliers': '#e74c3c',
    'Alto Rendimiento': '#27ae60',
    'Conservador': '#3498db',
    'Estable': '#9b59b6',
    'Moderado': '#f39c12'
}

colors = [colores_segmentos.get(s, '#95a5a6') for s in conteo_segmentos['segmento_nombre']]

fig_pie = go.Figure(data=[go.Pie(
    labels=conteo_segmentos['segmento_nombre'],
    values=conteo_segmentos['count'],
    hole=0.4,
    marker_colors=colors,
    textinfo='label+percent+value',
    textposition='outside'
)])

fig_pie.update_layout(
    title=dict(text='<b>Distribución de Activos por Segmento</b>', x=0.5, font=dict(size=18)),
    template='plotly_dark',
    height=500,
    showlegend=True
)

fig_pie.show()

In [19]:
# Tarjetas descriptivas de cada clúster/segmento
segmentos_info = {
    'Outliers': {
        'emoji': '⚠️',
        'descripcion': 'Activos con comportamiento atípico y alto riesgo. Volatilidad extrema y retornos impredecibles.',
        'caracteristicas': 'Alta volatilidad, Sharpe negativo, Max Drawdown severo',
        'recomendacion': 'Solo para perfiles especulativos con exposición limitada',
        'color': '#e74c3c'
    },
    'Alto Rendimiento': {
        'emoji': '🚀',
        'descripcion': 'Activos con los mejores retornos históricos. Alto potencial de crecimiento.',
        'caracteristicas': 'Retorno superior al mercado, Beta > 1, Volatilidad moderada-alta',
        'recomendacion': 'Ideal para perfiles agresivos y de crecimiento',
        'color': '#27ae60'
    },
    'Conservador': {
        'emoji': '🛡️',
        'descripcion': 'Activos defensivos con baja volatilidad. Protección en mercados bajistas.',
        'caracteristicas': 'Beta < 1, Baja volatilidad, Sharpe positivo',
        'recomendacion': 'Recomendado para preservación de capital',
        'color': '#3498db'
    },
    'Estable': {
        'emoji': '⚖️',
        'descripcion': 'Activos con comportamiento predecible y consistente. Bajo drawdown histórico.',
        'caracteristicas': 'Volatilidad baja, Max Drawdown controlado, Retornos moderados',
        'recomendacion': 'Base sólida para cualquier portafolio',
        'color': '#9b59b6'
    },
    'Moderado': {
        'emoji': '📊',
        'descripcion': 'Activos con balance riesgo-retorno equilibrado. Comportamiento similar al mercado.',
        'caracteristicas': 'Beta ≈ 1, Volatilidad media, Sharpe moderado',
        'recomendacion': 'Perfecto para diversificación y perfiles balanceados',
        'color': '#f39c12'
    }
}

# Generar HTML de tarjetas de segmentos
segmentos_html = '''
<div style="margin: 20px 0;">
    <h3 style="text-align: center; color: #00d4ff; margin-bottom: 20px;">🎯 DESCRIPCIÓN DE SEGMENTOS IDENTIFICADOS</h3>
    <div style="display: flex; flex-wrap: wrap; justify-content: center; gap: 15px;">
'''

for nombre, info in segmentos_info.items():
    segmentos_html += f'''
    <div style="background: linear-gradient(135deg, {info['color']}22, {info['color']}44); 
                border: 2px solid {info['color']}; border-radius: 15px; padding: 20px; 
                width: 300px;">
        <h3 style="margin: 0 0 10px 0; color: {info['color']}; text-align: center;">{info['emoji']} {nombre}</h3>
        <p style="font-size: 12px; color: #333; margin: 8px 0; text-align: justify;">{info['descripcion']}</p>
        <p style="font-size: 11px; color: #444; margin: 8px 0;"><b>📌 Características:</b> {info['caracteristicas']}</p>
        <p style="font-size: 11px; color: #555; margin: 8px 0 0 0; border-top: 1px solid {info['color']}44; padding-top: 8px;">
            <b>💡 Recomendación:</b> {info['recomendacion']}</p>
    </div>
    '''

segmentos_html += '</div></div>'

display(HTML(segmentos_html))

In [6]:
# Scatter Plot: Riesgo vs Retorno por Segmento
# Normalizar sharpe_ratio para que sea positivo (plotly size requiere valores >= 0)
df_segmentacion['sharpe_size'] = df_segmentacion['sharpe_ratio'] - df_segmentacion['sharpe_ratio'].min() + 0.1

fig_scatter = px.scatter(
    df_segmentacion,
    x='volatility_annual',
    y='return_annualized',
    color='segmento_nombre',
    size='sharpe_size',
    hover_data=['ticker', 'sharpe_ratio', 'beta', 'max_drawdown'],
    title='<b>Mapa Riesgo-Retorno por Segmento</b>',
    labels={
        'volatility_annual': 'Volatilidad Anualizada',
        'return_annualized': 'Retorno Anualizado',
        'segmento_nombre': 'Segmento'
    },
    color_discrete_map=colores_segmentos,
    template='plotly_dark'
)

fig_scatter.update_layout(
    height=600,
    xaxis_tickformat='.0%',
    yaxis_tickformat='.0%'
)

# Agregar líneas de referencia
fig_scatter.add_hline(y=0, line_dash='dash', line_color='gray', opacity=0.5)
fig_scatter.add_vline(x=df_segmentacion['volatility_annual'].mean(), line_dash='dash', line_color='gray', opacity=0.5)

fig_scatter.show()

## 5. Perfiles de Inversión y Composición de Carteras

In [ ]:
# Definición de perfiles
perfiles_info = {
    'Conservador': {
        'emoji': '🛡️',
        'descripcion': 'Prioriza la estabilidad y preservación del capital',
        'distribucion': '60% Estable, 20% Conservador, 20% Moderado',
        'color': '#3498db'
    },
    'Moderado': {
        'emoji': '⚖️',
        'descripcion': 'Balance entre crecimiento y estabilidad',
        'distribucion': '40% Alto Rendimiento, 30% Moderado, 30% Estable',
        'color': '#f39c12'
    },
    'Agresivo': {
        'emoji': '🚀',
        'descripcion': 'Busca alto rendimiento con exposición a beta alto',
        'distribucion': '70% Alto Rendimiento, 20% Moderado, 10% Outliers+',
        'color': '#e74c3c'
    },
    'Especulativo': {
        'emoji': '💎',
        'descripcion': 'Máximo rendimiento potencial con alta exposición a outliers positivos',
        'distribucion': '50% Alto Rendimiento, 30% Outliers+, 20% Moderado',
        'color': '#9b59b6'
    },
    'Normal': {
        'emoji': '📊',
        'descripcion': 'Exposición balanceada a todos los clusters (2 de cada uno)',
        'distribucion': '2 activos de cada segmento (incluyendo Outliers+)',
        'color': '#27ae60'
    }
}

# Mostrar tarjetas de perfiles
perfiles_html = '<div style="display: flex; flex-wrap: wrap; justify-content: center; gap: 15px;">'

for nombre, info in perfiles_info.items():
    perfiles_html += f'''
    <div style="background: linear-gradient(135deg, {info['color']}22, {info['color']}44); 
                border: 2px solid {info['color']}; border-radius: 15px; padding: 20px; 
                width: 280px; text-align: center;">
        <h3 style="margin: 0; color: {info['color']};">{info['emoji']} {nombre}</h3>
        <p style="font-size: 12px; color: #333; margin: 10px 0;">{info['descripcion']}</p>
        <p style="font-size: 11px; color: #444; margin: 0;"><b>Distribución:</b> {info['distribucion']}</p>
    </div>
    '''

perfiles_html += '</div>'

print("="*80)
print("🎯 PERFILES DE INVERSIÓN DISPONIBLES")
print("="*80)
display(HTML(perfiles_html))

🎯 PERFILES DE INVERSIÓN DISPONIBLES


In [8]:
# Tabla de composición de carteras para cada perfil
print("\n" + "="*80)
print("📋 COMPOSICIÓN DE CARTERAS POR PERFIL")
print("="*80)

for perfil_name, df_port in portafolios.items():
    print(f"\n{'─'*60}")
    print(f"📌 PORTAFOLIO {perfil_name.upper()}")
    print(f"{'─'*60}")
    
    # Mostrar composición
    cols_mostrar = ['ticker', 'segmento_nombre', 'return_annualized', 'volatility_annual', 'sharpe_ratio', 'beta', 'peso']
    cols_disponibles = [c for c in cols_mostrar if c in df_port.columns]
    
    df_display = df_port[cols_disponibles].copy()
    
    if 'return_annualized' in df_display.columns:
        df_display['return_annualized'] = df_display['return_annualized'].apply(lambda x: f"{x*100:.2f}%")
    if 'volatility_annual' in df_display.columns:
        df_display['volatility_annual'] = df_display['volatility_annual'].apply(lambda x: f"{x*100:.2f}%")
    if 'sharpe_ratio' in df_display.columns:
        df_display['sharpe_ratio'] = df_display['sharpe_ratio'].apply(lambda x: f"{x:.2f}")
    if 'beta' in df_display.columns:
        df_display['beta'] = df_display['beta'].apply(lambda x: f"{x:.2f}")
    if 'peso' in df_display.columns:
        df_display['peso'] = df_display['peso'].apply(lambda x: f"{x*100:.1f}%")
    
    display(df_display)


📋 COMPOSICIÓN DE CARTERAS POR PERFIL

────────────────────────────────────────────────────────────
📌 PORTAFOLIO CONSERVADOR
────────────────────────────────────────────────────────────


,ticker,segmento_nombre,return_annualized,volatility_annual,sharpe_ratio,beta,peso
0,GS,Estable,13.42%,26.44%,0.42,0.98,10.0%
1,BRO,Estable,15.44%,24.95%,0.50,0.90,10.0%
2,NRG,Estable,12.59%,34.08%,0.37,0.84,10.0%
3,ACN,Estable,11.09%,25.63%,0.34,1.15,10.0%
4,NWSA,Estable,10.64%,30.75%,0.32,1.05,10.0%
5,QQQ,Estable,9.87%,23.76%,0.30,1.26,10.0%
6,EWG,Conservador,-2.64%,21.91%,-0.24,0.93,10.0%
7,C,Conservador,-5.11%,28.49%,-0.22,0.98,10.0%
8,AMAT,Moderado,23.26%,43.41%,0.58,1.77,10.0%
9,RCL,Moderado,22.49%,52.29%,0.55,1.63,10.0%



────────────────────────────────────────────────────────────
📌 PORTAFOLIO MODERADO
────────────────────────────────────────────────────────────


,ticker,segmento_nombre,return_annualized,volatility_annual,sharpe_ratio,beta,peso
0,NUE,Alto_Rendimiento,49.47%,40.80%,1.07,1.19,10.0%
1,PWR,Alto_Rendimiento,47.12%,33.16%,1.18,1.08,10.0%
2,IT,Alto_Rendimiento,42.15%,31.24%,1.12,1.09,10.0%
3,OXY,Alto_Rendimiento,51.07%,48.81%,0.98,0.89,10.0%
4,AMAT,Moderado,23.26%,43.41%,0.58,1.77,10.0%
5,RCL,Moderado,22.49%,52.29%,0.55,1.63,10.0%
6,AMD,Moderado,17.00%,50.92%,0.46,1.96,10.0%
7,GS,Estable,13.42%,26.44%,0.42,0.98,10.0%
8,BRO,Estable,15.44%,24.95%,0.50,0.90,10.0%
9,NRG,Estable,12.59%,34.08%,0.37,0.84,10.0%



────────────────────────────────────────────────────────────
📌 PORTAFOLIO AGRESIVO
────────────────────────────────────────────────────────────


,ticker,segmento_nombre,return_annualized,volatility_annual,sharpe_ratio,beta,peso
0,NUE,Alto_Rendimiento,49.47%,40.80%,1.07,1.19,10.0%
1,PWR,Alto_Rendimiento,47.12%,33.16%,1.18,1.08,10.0%
2,IT,Alto_Rendimiento,42.15%,31.24%,1.12,1.09,10.0%
3,OXY,Alto_Rendimiento,51.07%,48.81%,0.98,0.89,10.0%
4,FANG,Alto_Rendimiento,47.23%,44.57%,0.98,0.94,10.0%
5,AVGO,Alto_Rendimiento,38.23%,32.88%,1.00,1.35,10.0%
6,URI,Alto_Rendimiento,35.92%,38.44%,0.86,1.42,10.0%
7,AMAT,Moderado,23.26%,43.41%,0.58,1.77,10.0%
8,RCL,Moderado,22.49%,52.29%,0.55,1.63,10.0%
9,NVDA,Outliers_Riesgo_Extremo,56.17%,52.98%,1.01,2.13,10.0%



────────────────────────────────────────────────────────────
📌 PORTAFOLIO ESPECULATIVO
────────────────────────────────────────────────────────────


,ticker,segmento_nombre,return_annualized,volatility_annual,sharpe_ratio,beta,peso
0,NUE,Alto_Rendimiento,49.47%,40.80%,1.07,1.19,10.0%
1,PWR,Alto_Rendimiento,47.12%,33.16%,1.18,1.08,10.0%
2,IT,Alto_Rendimiento,42.15%,31.24%,1.12,1.09,10.0%
3,OXY,Alto_Rendimiento,51.07%,48.81%,0.98,0.89,10.0%
4,FANG,Alto_Rendimiento,47.23%,44.57%,0.98,0.94,10.0%
5,NVDA,Outliers_Riesgo_Extremo,56.17%,52.98%,1.01,2.13,10.0%
6,MPC,Outliers_Riesgo_Extremo,54.25%,33.01%,1.33,0.78,10.0%
7,LLY,Outliers_Riesgo_Extremo,52.55%,29.27%,1.42,0.52,10.0%
8,AMAT,Moderado,23.26%,43.41%,0.58,1.77,10.0%
9,RCL,Moderado,22.49%,52.29%,0.55,1.63,10.0%



────────────────────────────────────────────────────────────
📌 PORTAFOLIO NORMAL
────────────────────────────────────────────────────────────


,ticker,segmento_nombre,return_annualized,volatility_annual,sharpe_ratio,beta,peso
0,NVDA,Outliers_Riesgo_Extremo,56.17%,52.98%,1.01,2.13,10.0%
1,MPC,Outliers_Riesgo_Extremo,54.25%,33.01%,1.33,0.78,10.0%
2,EWG,Conservador,-2.64%,21.91%,-0.24,0.93,10.0%
3,C,Conservador,-5.11%,28.49%,-0.22,0.98,10.0%
4,NUE,Alto_Rendimiento,49.47%,40.80%,1.07,1.19,10.0%
5,PWR,Alto_Rendimiento,47.12%,33.16%,1.18,1.08,10.0%
6,AMAT,Moderado,23.26%,43.41%,0.58,1.77,10.0%
7,RCL,Moderado,22.49%,52.29%,0.55,1.63,10.0%
8,GS,Estable,13.42%,26.44%,0.42,0.98,10.0%
9,BRO,Estable,15.44%,24.95%,0.50,0.90,10.0%


## 6. Resultados del Backtesting

In [9]:
# Consolidar métricas de todos los perfiles
print("="*80)
print("📊 MÉTRICAS COMPARATIVAS DE BACKTESTING")
print("="*80)

metricas_consolidadas = []

for perfil_name, df_metricas in metricas_backtest.items():
    # Separar métricas de portafolio y benchmark
    for _, row in df_metricas.iterrows():
        metricas_consolidadas.append({
            'Perfil': perfil_name.capitalize(),
            'Métrica': row.iloc[0],
            'Portafolio': row.iloc[1] if len(row) > 1 else None,
            'Benchmark': row.iloc[2] if len(row) > 2 else None
        })

df_metricas_all = pd.DataFrame(metricas_consolidadas)

# Crear tabla pivote para comparación
if len(metricas_backtest) > 0:
    # Mostrar métricas clave por perfil
    metricas_clave = []
    
    for perfil_name, df_metricas in metricas_backtest.items():
        try:
            # El CSV usa 'Metrica' como columna y nombres con guion bajo
            # Formato: Metrica, Portafolio_[Perfil], SPY_Benchmark
            df_metricas.columns = ['Metrica', 'Portafolio', 'Benchmark']
            
            retorno_port = df_metricas[df_metricas['Metrica'] == 'Retorno_Total']['Portafolio'].values[0]
            retorno_bench = df_metricas[df_metricas['Metrica'] == 'Retorno_Total']['Benchmark'].values[0]
            sharpe_port = df_metricas[df_metricas['Metrica'] == 'Sharpe_Ratio']['Portafolio'].values[0]
            max_dd_port = df_metricas[df_metricas['Metrica'] == 'Max_Drawdown']['Portafolio'].values[0]
            
            # Los valores ya están en formato decimal (e.g., 0.157 = 15.7%)
            alpha = retorno_port - retorno_bench
            
            metricas_clave.append({
                'Perfil': perfil_name.capitalize(),
                'Retorno Portafolio': retorno_port,
                'Retorno SPY': retorno_bench,
                'Alpha': alpha,
                'Sharpe Ratio': sharpe_port,
                'Max Drawdown': max_dd_port
            })
        except Exception as e:
            print(f"⚠️ Error procesando {perfil_name}: {e}")
    
    df_resumen = pd.DataFrame(metricas_clave)
    
    # Formatear y mostrar
    if len(df_resumen) > 0:
        df_resumen_display = df_resumen.copy()
        df_resumen_display['Retorno Portafolio'] = df_resumen_display['Retorno Portafolio'].apply(lambda x: f"{x*100:.2f}%")
        df_resumen_display['Retorno SPY'] = df_resumen_display['Retorno SPY'].apply(lambda x: f"{x*100:.2f}%")
        df_resumen_display['Alpha'] = df_resumen_display['Alpha'].apply(lambda x: f"{x*100:+.2f}%")
        df_resumen_display['Sharpe Ratio'] = df_resumen_display['Sharpe Ratio'].apply(lambda x: f"{x:.3f}")
        df_resumen_display['Max Drawdown'] = df_resumen_display['Max Drawdown'].apply(lambda x: f"{x*100:.2f}%")
        
        display(df_resumen_display)
    else:
        print("⚠️ No se pudieron procesar las métricas")

📊 MÉTRICAS COMPARATIVAS DE BACKTESTING


,Perfil,Retorno Portafolio,Retorno SPY,Alpha,Sharpe Ratio,Max Drawdown
0,Conservador,76.00%,43.84%,+32.16%,1.346,-22.76%
1,Moderado,63.23%,43.84%,+19.39%,0.963,-24.02%
2,Agresivo,74.85%,43.84%,+31.01%,0.985,-30.93%
3,Especulativo,58.76%,43.84%,+14.92%,0.881,-28.04%
4,Normal,89.32%,43.84%,+45.49%,1.416,-23.66%


In [10]:
# Gráfico de barras: Alpha por Perfil
if len(metricas_clave) > 0:
    df_alpha = pd.DataFrame(metricas_clave)
    
    # Ordenar por alpha
    df_alpha = df_alpha.sort_values('Alpha', ascending=True)
    
    colors_bar = ['#e74c3c' if x < 0 else '#27ae60' for x in df_alpha['Alpha']]
    
    fig_alpha = go.Figure(data=[
        go.Bar(
            x=df_alpha['Alpha'] * 100,
            y=df_alpha['Perfil'],
            orientation='h',
            marker_color=colors_bar,
            text=[f"{x*100:+.2f}%" for x in df_alpha['Alpha']],
            textposition='outside'
        )
    ])
    
    fig_alpha.update_layout(
        title=dict(text='<b>Alpha Generado por Perfil de Inversión vs SPY</b>', x=0.5, font=dict(size=18)),
        xaxis_title='Alpha (%)',
        yaxis_title='Perfil',
        template='plotly_dark',
        height=400,
        xaxis=dict(zeroline=True, zerolinecolor='white', zerolinewidth=2)
    )
    
    fig_alpha.show()

In [11]:
# Gráfico de Equity Curves consolidado
print("\n" + "="*80)
print("📈 CURVAS DE EQUITY COMPARATIVAS")
print("="*80)

fig_equity = go.Figure()

colores_perfiles = {
    'conservador': '#3498db',
    'moderado': '#f39c12',
    'agresivo': '#e74c3c',
    'especulativo': '#9b59b6',
    'normal': '#27ae60'
}

benchmark_agregado = False

for perfil_name, df_equity in equity_curves.items():
    # Agregar equity del portafolio
    if 'equity_portafolio' in df_equity.columns:
        fig_equity.add_trace(go.Scatter(
            x=df_equity.index,
            y=df_equity['equity_portafolio'],
            mode='lines',
            name=f'Portafolio {perfil_name.capitalize()}',
            line=dict(color=colores_perfiles.get(perfil_name, '#888'), width=2)
        ))
    
    # Agregar benchmark solo una vez
    if not benchmark_agregado and 'equity_benchmark' in df_equity.columns:
        fig_equity.add_trace(go.Scatter(
            x=df_equity.index,
            y=df_equity['equity_benchmark'],
            mode='lines',
            name='SPY (Benchmark)',
            line=dict(color='white', width=3, dash='dash')
        ))
        benchmark_agregado = True

fig_equity.update_layout(
    title=dict(text='<b>Evolución del Capital: Todos los Perfiles vs SPY</b>', x=0.5, font=dict(size=18)),
    xaxis_title='Fecha',
    yaxis_title='Valor del Portafolio ($)',
    template='plotly_dark',
    height=600,
    hovermode='x unified',
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='center', x=0.5),
    yaxis=dict(range=[8000, 22000])  # Ajustar rango para mejor visualización
)

# Agregar línea de capital inicial
fig_equity.add_hline(y=10000, line_dash='dot', line_color='gray', opacity=0.5, 
                     annotation_text='Capital Inicial ($10,000)', annotation_position='bottom right')

fig_equity.show()


📈 CURVAS DE EQUITY COMPARATIVAS


## 7. Análisis Comparativo de Perfiles

In [12]:
# Radar Chart comparativo
if len(metricas_clave) > 0:
    # Normalizar métricas para el radar (0-1)
    df_radar = pd.DataFrame(metricas_clave)
    
    # Métricas para el radar (invertir Max Drawdown para que mayor sea mejor)
    df_radar['Max DD Inv'] = 1 + df_radar['Max Drawdown']  # Convertir a positivo (menos negativo = mejor)
    
    # Normalizar cada métrica
    for col in ['Retorno Portafolio', 'Alpha', 'Sharpe Ratio', 'Max DD Inv']:
        min_val = df_radar[col].min()
        max_val = df_radar[col].max()
        if max_val - min_val > 0:
            df_radar[f'{col}_norm'] = (df_radar[col] - min_val) / (max_val - min_val)
        else:
            df_radar[f'{col}_norm'] = 0.5
    
    categories = ['Retorno', 'Alpha', 'Sharpe', 'Protección DD']
    
    fig_radar = go.Figure()
    
    for _, row in df_radar.iterrows():
        valores = [
            row['Retorno Portafolio_norm'],
            row['Alpha_norm'],
            row['Sharpe Ratio_norm'],
            row['Max DD Inv_norm']
        ]
        valores.append(valores[0])  # Cerrar el polígono
        
        fig_radar.add_trace(go.Scatterpolar(
            r=valores,
            theta=categories + [categories[0]],
            fill='toself',
            name=row['Perfil'],
            opacity=0.6,
            line=dict(color=colores_perfiles.get(row['Perfil'].lower(), '#888'))
        ))
    
    fig_radar.update_layout(
        polar=dict(
            radialaxis=dict(visible=True, range=[0, 1])
        ),
        title=dict(text='<b>Comparación de Perfiles: Análisis Multidimensional</b>', x=0.5, font=dict(size=18)),
        template='plotly_dark',
        height=500,
        showlegend=True
    )
    
    fig_radar.show()

## 8. Fórmula de Selección de Activos

In [13]:
formula_html = """
<div style="background: linear-gradient(135deg, #0f0f23 0%, #1a1a3e 100%); padding: 30px; border-radius: 15px; color: white; font-family: 'Courier New', monospace;">
    <h2 style="text-align: center; color: #00d4ff; margin-bottom: 25px;">🧮 FÓRMULA DE SELECCIÓN DE ACTIVOS</h2>
    
    <div style="background: rgba(0,212,255,0.1); padding: 25px; border-radius: 10px; text-align: center; margin-bottom: 20px;">
        <h3 style="color: #00ff88; font-size: 24px;">Score = 0.35×Return + 0.30×Momentum₆ₘ + 0.15×Sharpe + 0.20×Beta</h3>
    </div>
    
    <h4 style="color: #f39c12; margin-top: 25px;">📊 Componentes del Score:</h4>
    <table style="width: 100%; border-collapse: collapse; margin-top: 15px;">
        <tr style="background: rgba(255,255,255,0.1);">
            <th style="padding: 12px; text-align: left; border-bottom: 2px solid #00d4ff;">Componente</th>
            <th style="padding: 12px; text-align: center; border-bottom: 2px solid #00d4ff;">Peso</th>
            <th style="padding: 12px; text-align: left; border-bottom: 2px solid #00d4ff;">Descripción</th>
        </tr>
        <tr>
            <td style="padding: 10px; color: #00ff88;">Return (Retorno Anualizado)</td>
            <td style="padding: 10px; text-align: center;">35%</td>
            <td style="padding: 10px;">Prioriza rendimiento absoluto histórico</td>
        </tr>
        <tr style="background: rgba(255,255,255,0.05);">
            <td style="padding: 10px; color: #00ff88;">Momentum 6m</td>
            <td style="padding: 10px; text-align: center;">30%</td>
            <td style="padding: 10px;">Captura tendencias recientes (factor momentum)</td>
        </tr>
        <tr>
            <td style="padding: 10px; color: #00ff88;">Sharpe Ratio</td>
            <td style="padding: 10px; text-align: center;">15%</td>
            <td style="padding: 10px;">Eficiencia riesgo-retorno</td>
        </tr>
        <tr style="background: rgba(255,255,255,0.05);">
            <td style="padding: 10px; color: #00ff88;">Beta</td>
            <td style="padding: 10px; text-align: center;">20%</td>
            <td style="padding: 10px;">En mercados alcistas, beta alto amplifica ganancias</td>
        </tr>
    </table>
    
    <div style="background: rgba(231,76,60,0.2); padding: 15px; border-radius: 8px; margin-top: 20px; border-left: 4px solid #e74c3c;">
        <p style="margin: 0; color: #e74c3c;"><strong>⚠️ Nota sobre Outliers:</strong> Los activos del cluster de Outliers (-1) solo se incluyen si tienen rendimiento positivo, para evitar activos de alto riesgo con mal desempeño.</p>
    </div>
</div>
"""

display(HTML(formula_html))

Componente,Peso,Descripción
Return (Retorno Anualizado),35%,Prioriza rendimiento absoluto histórico
Momentum 6m,30%,Captura tendencias recientes (factor momentum)
Sharpe Ratio,15%,Eficiencia riesgo-retorno
Beta,20%,"En mercados alcistas, beta alto amplifica ganancias"


## 9. Conclusiones y Recomendaciones

In [14]:
conclusiones_html = """
<div style="background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%); padding: 30px; border-radius: 15px; color: white;">
    <h2 style="text-align: center; color: #00ff88; margin-bottom: 25px;">✅ CONCLUSIONES</h2>
    
    <div style="display: grid; grid-template-columns: repeat(auto-fit, minmax(300px, 1fr)); gap: 20px;">
        <div style="background: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px;">
            <h4 style="color: #00d4ff; margin-top: 0;">1. Efectividad del Clustering</h4>
            <p>La segmentación mediante K-Means + DBSCAN identificó exitosamente grupos de activos con características de riesgo-retorno diferenciadas, permitiendo una diversificación efectiva.</p>
        </div>
        
        <div style="background: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px;">
            <h4 style="color: #00d4ff; margin-top: 0;">2. Generación de Alpha</h4>
            <p>Las carteras construidas con la metodología de Score de Momentum lograron superar al benchmark SPY en el período de prueba, validando la estrategia de selección.</p>
        </div>
        
        <div style="background: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px;">
            <h4 style="color: #00d4ff; margin-top: 0;">3. Factor Beta</h4>
            <p>En mercados alcistas, la exposición a activos de alto beta resultó beneficiosa. La corrección de la fórmula (beta positivo) fue clave para capturar el rally de 2024-2025.</p>
        </div>
        
        <div style="background: rgba(255,255,255,0.1); padding: 20px; border-radius: 10px;">
            <h4 style="color: #00d4ff; margin-top: 0;">4. Perfiles Diferenciados</h4>
            <p>Los 5 perfiles de inversión ofrecen opciones claras para diferentes tolerancias al riesgo, desde conservador hasta especulativo, con resultados consistentes.</p>
        </div>
    </div>
    
    <h2 style="text-align: center; color: #f39c12; margin: 30px 0 20px 0;">📋 RECOMENDACIONES Y PASOS SIGUIENTES</h2>
    
    <div style="background: rgba(243,156,18,0.1); padding: 20px; border-radius: 10px;">
        <ol style="margin: 0; padding-left: 20px;">
            <li style="margin-bottom: 12px;"><strong>Estrategias de Rebalanceo:</strong> Implementar rebalanceo periódico (mensual/trimestral) para mantener la distribución objetivo y capturar ganancias.</li>
            <li style="margin-bottom: 12px;"><strong>Actualización de Modelos:</strong> Re-entrenar el clustering con datos hasta 2025 para obtener segmentaciones actualizadas.</li>
            <li style="margin-bottom: 12px;"><strong>Optimización de Pesos:</strong> Explorar métodos de optimización de portafolio (Mean-Variance, Risk Parity) además de la equiponderación.</li>
            <li style="margin-bottom: 12px;"><strong>Factores Adicionales:</strong> Incorporar factores macroeconómicos y de sentimiento para mejorar la selección.</li>
            <li style="margin-bottom: 12px;"><strong>Producción:</strong> Desarrollar una aplicación web para que los usuarios puedan seleccionar su perfil y obtener recomendaciones personalizadas.</li>
        </ol>
    </div>
</div>
"""

display(HTML(conclusiones_html))

## 10. Disclaimer Legal

In [15]:
disclaimer_html = """
<div style="background: #2c2c2c; padding: 30px; border-radius: 15px; border: 2px solid #e74c3c; color: #ccc;">
    <h2 style="text-align: center; color: #e74c3c; margin-bottom: 20px;">⚠️ AVISO LEGAL - DISCLAIMER</h2>
    
    <div style="text-align: justify; line-height: 1.8;">
        <p><strong style="color: #e74c3c;">ESTE PROYECTO NO CONSTITUYE ASESORÍA DE INVERSIÓN</strong></p>
        
        <p>De conformidad con lo establecido en el Decreto 2555 de 2010 y la Ley 964 de 2005 del mercado de valores colombiano, 
        se hace constar expresamente que:</p>
        
        <ul style="margin: 15px 0;">
            <li>Este proyecto es de carácter <strong>exclusivamente personal, educativo y de investigación</strong>.</li>
            <li>Los análisis, resultados y recomendaciones presentados <strong>NO constituyen asesoría financiera, 
            recomendación de inversión, ni oferta de productos o servicios financieros</strong>.</li>
            <li>El autor <strong>NO es un asesor de inversión autorizado</strong> por la Superintendencia Financiera de Colombia.</li>
            <li>Los rendimientos pasados <strong>NO garantizan resultados futuros</strong>.</li>
            <li>Toda decisión de inversión debe ser tomada con el acompañamiento de un 
            <strong>profesional autorizado y regulado</strong> por las autoridades competentes.</li>
        </ul>
        
        <p style="background: rgba(231,76,60,0.2); padding: 15px; border-radius: 8px;">
        <strong>ADVERTENCIA DE RIESGO:</strong> Las inversiones en el mercado de valores conllevan riesgos significativos, 
        incluyendo la posible pérdida total del capital invertido. Cada inversionista debe evaluar su perfil de riesgo, 
        situación financiera y objetivos antes de tomar cualquier decisión de inversión.</p>
        
        <p style="margin-top: 20px; font-size: 12px; color: #888;">
        <em>El autor no se hace responsable por pérdidas, daños o perjuicios que puedan derivarse del uso de la información 
        contenida en este proyecto. El uso de esta información es bajo la exclusiva responsabilidad del usuario.</em></p>
    </div>
    
    <hr style="border-color: #444; margin: 25px 0;">
    
    <p style="text-align: center; font-size: 14px;">
        <strong>Proyecto desarrollado por:</strong> Juan Carlos Ruiz Arteaga<br>
        <strong>Contacto:</strong> carlosarte11@gmail.com<br>
        <strong>Fecha:</strong> Enero 2026<br>
        <strong>Licencia:</strong> MIT License
    </p>
</div>
"""

display(HTML(disclaimer_html))

## 11. Exportar Reporte Final

In [16]:
# Crear resumen ejecutivo en CSV
print("="*80)
print("💾 EXPORTANDO REPORTE FINAL")
print("="*80)

# 1. Resumen de métricas por perfil
if 'df_resumen' in dir() and len(df_resumen) > 0:
    df_resumen.to_csv(f'{PATH_REPORTS}/reporte_final_metricas.csv', index=False)
    print(f"✅ Métricas exportadas: reporte_final_metricas.csv")

# 2. Resumen de segmentos
segmentos_resumen.to_csv(f'{PATH_REPORTS}/reporte_final_segmentos.csv', index=False)
print(f"✅ Segmentos exportados: reporte_final_segmentos.csv")

# 3. Crear archivo de resumen general
resumen_general = {
    'Métrica': [
        'Total Activos Analizados',
        'Período Entrenamiento',
        'Período Prueba',
        'Número de Segmentos',
        'Número de Perfiles',
        'Benchmark',
        'Capital Inicial Backtest',
        'Costos de Transacción'
    ],
    'Valor': [
        len(df_segmentacion),
        f"{df_precios_train.index.min().strftime('%Y-%m-%d')} a {df_precios_train.index.max().strftime('%Y-%m-%d')}",
        f"{df_precios_test.index.min().strftime('%Y-%m-%d')} a {df_precios_test.index.max().strftime('%Y-%m-%d')}",
        5,
        5,
        'SPY (S&P 500)',
        '$100,000',
        '0.10% round-trip'
    ]
}

pd.DataFrame(resumen_general).to_csv(f'{PATH_REPORTS}/reporte_final_resumen.csv', index=False)
print(f"✅ Resumen general exportado: reporte_final_resumen.csv")

print(f"\n📁 Archivos guardados en: {PATH_REPORTS}")
print("\n" + "="*80)
print("🎉 REPORTE FINAL COMPLETADO")
print("="*80)

💾 EXPORTANDO REPORTE FINAL
✅ Métricas exportadas: reporte_final_metricas.csv
✅ Segmentos exportados: reporte_final_segmentos.csv
✅ Resumen general exportado: reporte_final_resumen.csv

📁 Archivos guardados en: C:\Users\carlo\Documents\4.DS\riskmanagement2025\reports

🎉 REPORTE FINAL COMPLETADO
